In [2]:
from azureml.core import Workspace, Experiment

print("Accessing the workspace from job...")
ws = Workspace.from_config("./config")

print("Accessing the Defaults dataset...")
input_ds = ws.datasets.get("Defaults")

Accessing the workspace from job...
Accessing the Defaults dataset...


In [3]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

myenv = Environment(name="Myenv")
myenv_dep = CondaDependencies.create(conda_packages=['scikit-learn', "pip"],
                                     pip_packages=['azureml-defaults'])
myenv.python.conda_dependencies = myenv_dep

print("Registering the environmenr...")
myenv.register(ws)

Registering the environmenr...


{
    "assetId": "azureml://locations/westus/workspaces/54b058d1-e724-4986-aece-5eb3256cb2e6/environments/Myenv/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20240304.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "Myenv",
    "python": {
        

In [4]:
cluster_name = "my-cluster-001"
from azureml.core.compute import AmlCompute

print("Accessing the compute cluster...")

if cluster_name not in ws.compute_targets:
    print("Creating the compute cluster with name: ", cluster_name)
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D11_V2", max_nodes=2)
    cluster = AmlCompute.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion()
else:
    cluster = ws.compute_targets[cluster_name]
    print(cluster_name, ", compute cluster found. Using it...")

Accessing the compute cluster...
Creating the compute cluster with name:  my-cluster-001
Provisioning operation finished, operation "Succeeded"


In [5]:
from azureml.core import ScriptRunConfig

script_config = ScriptRunConfig(source_directory=".",
                                script="320 - hyperdrive script.py",
                                arguments = ["--input-data", input_ds.as_named_input("raw_data")],
                                environment=myenv,
                                compute_target=cluster)

In [6]:
# Create Hyper drive parameters

from azureml.train.hyperdrive import GridParameterSampling, choice, HyperDriveConfig, PrimaryMetricGoal

hyper_params = GridParameterSampling({"--n_estimators": choice(10, 20, 50, 100),
                                      "--min_samples_leaf": choice(1, 2, 5)})

# Configure Hyperdrive class

hyper_config = HyperDriveConfig(run_config=script_config,
                                hyperparameter_sampling=hyper_params,
                                policy=None,
                                primary_metric_name="accuracy",
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=20,
                                max_concurrent_runs=2)

In [ ]:
new_experiment = Experiment(workspace=ws, name="HyperExp1")
new_run = new_experiment.submit(config=hyper_config)
new_run.wait_for_completion(show_output=True)